# Quantized SNN for MNIST

## Imports

In [ ]:
# SNN
import snntorch as snn
from snntorch import surrogate
from snntorch import functional as SF
from snntorch.functional import quant
from snntorch import utils
from snntorch import spikeplot as splt
from snntorch import spikegen

# Quantization
import brevitas.nn as qnn

# Torch
import torch
from torch import nn
from torch.nn import Module
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Other
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

### Network Definition

In [ ]:
config = {
    "num_epochs": 1,       # Number of epochs to train for (per trial)
    "batch_size": 128,      # Batch size
    "seed": 0,              # Random seed
    
    # Quantization
    "num_bits": 4,          # Bit resolution
    
    # Network parameters
    "grad_clip": False,     # Whether or not to clip gradients
    "weight_clip": False,   # Whether or not to clip weights
    "batch_norm": True,     # Whether or not to use batch normalization
    "dropout": 0.07,        # Dropout rate
    "beta": 1.00,           # Decay rate parameter (beta)
    "threshold": 30,        # Threshold parameter (theta)
    "lr": 3.0e-3,           # Initial learning rate
    "slope": 5.6,           # Slope value (k)
    
    # Fixed params
    "num_steps": 100,       # Number of timesteps to encode input for
    "correct_rate": 0.8,    # Correct rate
    "incorrect_rate": 0.2,  # Incorrect rate
    "betas": (0.9, 0.999),  # Adam optimizer beta values
    "t_0": 4690,            # Initial frequency of the cosine annealing scheduler
    "eta_min": 0,           # Minimum learning rate
}

In [ ]:
device = torch.device("cpu")

class Net(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_bits   = config["num_bits"]
        self.thr        = config["threshold"]
        self.slope      = config["slope"]
        self.beta       = config["beta"]
        self.num_steps  = config["num_steps"]
        
        # Initialize Layers
        self.fc1        = qnn.QuantLinear(9, 3, bias=False, weight_bit_width=self.num_bits)
        self.lif1       = snn.Leaky(self.beta, threshold=self.thr)

    def forward(self, x):
        # Initialize hidden states and outputs at t=0
        mem = self.lif1.init_leaky()
        
        # Record the final layer
        spk_rec = []
        mem_rec = []
        for step in range(self.num_steps):
            cur = self.fc1(x[step])
            spk, mem = self.lif1(cur, mem)

            spk_rec.append(spk)
            mem_rec.append(mem)
        
        return torch.stack(spk_rec), torch.stack(mem_rec)

net = Net(config).to(device)

In [ ]:
with torch.no_grad():
    manual_weights = torch.ones((3, 9), device=net.fc1.weight.device)
    net.fc1.weight.copy_(manual_weights)

In [ ]:
nrn_idx_list = [0, 1, 2]
weights_list = [0.25, -0.5, 0.5]
with torch.no_grad():
    for nrn_idx, weight in zip(nrn_idx_list, weights_list):
        net.fc1.weight[nrn_idx, :] = weight

In [ ]:
net.lif1.threshold/net.fc1.quant_weight().scale

In [ ]:
# This seems to work
net.fc1.quant_weight()/net.fc1.quant_weight().scale

In [ ]:
timesteps   = 100
batch_size  = 1
channels    = 1
length      = 9

data = torch.zeros((timesteps, batch_size, length))
data[:, :, 0] = 1.0

spk_rec, mem_rec = net(data)
spk_rec = spk_rec.detach().numpy()
mem_rec = mem_rec.detach().numpy()

In [ ]:
x = np.arange(0, 100, 1)

fig, ax = plt.subplots(3, 2, figsize=(16, 6))
ax[0, 0].plot(x, mem_rec[:, 0, 0], label="nrn = 0")
ax[1, 0].plot(x, mem_rec[:, 0, 1], label="nrn = 1")
ax[2, 0].plot(x, mem_rec[:, 0, 2], label="nrn = 2")
ax[0, 1].plot(x, spk_rec[:, 0, 0], label="nrn = 0")
ax[1, 1].plot(x, spk_rec[:, 0, 1], label="nrn = 1")
ax[2, 1].plot(x, spk_rec[:, 0, 2], label="nrn = 2")

ax[0, 0].set_title("mem_rec")
ax[0, 1].set_title("spk_rec")

for a in ax.flatten():
    a.legend(loc="upper left")

fig.suptitle("snntorch")

## Description

### `tb2`

```py
nrn_idx_list = [0, 1, 2]
weights_list = [0.25, -0.5, 0.5]
with torch.no_grad():
    for nrn_idx, weight in zip(nrn_idx_list, weights_list):
        net.fc1.weight[nrn_idx, :] = weight
```

### `tb1`

- 9 to 3 fc
- first weight is 0, other are 1 (7)

first line in `syn_init.data`
```
01110111011101110111011101110000
```

- threshold is 30 (210)

### `tb0`

- 9 to 3 fc
- all weights are 1 (7)
- threshold is 30 (210)

In [ ]:
def bin_to_signed_int(bstr, bits=12):
    """Convert binary string to signed integer."""
    val = int(bstr, 2)
    if val >= 2**(bits - 1):
        val -= 2**bits
    return val

df = pd.read_csv('data/testbench/tb2_mem_rec.csv', names=['nrnwrt_mem_addr', 'nrnwrt_mem_data'])
tb_mem_rec = df['nrnwrt_mem_data'].to_numpy()
tb_mem_rec_2 = [s[0:12] for s in tb_mem_rec]
tb_mem_rec_1 = [s[12:24] for s in tb_mem_rec]
tb_mem_rec_0 = [s[24:36] for s in tb_mem_rec]
tb_mem_rec_0_int = [bin_to_signed_int(b, bits=12) for b in tb_mem_rec_0]
tb_mem_rec_1_int = [bin_to_signed_int(b, bits=12) for b in tb_mem_rec_1]
tb_mem_rec_2_int = [bin_to_signed_int(b, bits=12) for b in tb_mem_rec_2]

x = np.arange(0, 100, 1)

df = pd.read_csv('data/testbench/tb2_spk_rec.csv', names=['event_number', 'out_fifo_wdata'])
df['out_fifo_wdata'] = df['out_fifo_wdata'].apply(lambda b: int(str(b), 2))

x = np.arange(0, 100, 1)
nrn_0 = np.zeros_like(x)
nrn_1 = np.zeros_like(x)
nrn_2 = np.zeros_like(x)

for event, nrn in zip(df['event_number'], df['out_fifo_wdata']):
    if nrn == 0:
        nrn_0[event] = 1
    elif nrn == 1:
        nrn_1[event] = 1
    elif nrn == 2:
        nrn_2[event] = 1

fig, ax = plt.subplots(3, 2, figsize=(16, 6))
ax[0, 0].plot(x, tb_mem_rec_0_int, label="nrn = 0")
ax[1, 0].plot(x, tb_mem_rec_1_int, label="nrn = 1")
ax[2, 0].plot(x, tb_mem_rec_2_int, label="nrn = 2")
ax[0, 1].plot(x, nrn_0, label="nrn = 0")
ax[1, 1].plot(x, nrn_1, label="nrn = 1")
ax[2, 1].plot(x, nrn_2, label="nrn = 2")

ax[0, 0].set_title("mem_rec")
ax[0, 1].set_title("spk_rec")

for a in ax.flatten():
    a.legend(loc="upper left")

fig.suptitle("testbench")